In [1]:
#https://query2.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries/AAPL?symbol=AAPL"
#https://query2.finance.yahoo.com/v8/finance/chart/AAPL?period1=1704145844&period2=1723236317&interval=1mo

In [1]:
import sys
import os
cwd = os.getcwd()

sys.path.append(cwd.replace('notebooks','')+'code')

In [2]:
from source.scraper import generate_CIK_TICKER


df = generate_CIK_TICKER()

In [7]:
from spark_deltalake.spark_operations import sparkDelta


spark = sparkDelta()
pydf = spark.get_spark_dataframe('companyFacts', True)

hadoop-azure is available
azure-storage is available


In [8]:
pydf.show()

+--------------------+----------+----------+--------------------+---+-----+----------+----------+---------+--------+------+------------+--------------------+---------------------+--------------------+------------+
|     accessionNumber|filingDate|reportDate|  acceptanceDateTime|act| form|fileNumber|filmNumber|    items|    size|isXBRL|isInlineXBRL|     primaryDocument|primaryDocDescription|             fileURL|yearMonthDay|
+--------------------+----------+----------+--------------------+---+-----+----------+----------+---------+--------+------+------------+--------------------+---------------------+--------------------+------------+
|0000320193-19-000100|2019-10-01|2019-09-29|2019-10-01T18:31:...|   |    4|          |          |         |    5671|     0|           0|xslF345X03/wf-for...|               FORM 4|https://www.sec.g...|    20190929|
|0000320193-19-000099|2019-10-01|2019-09-29|2019-10-01T18:30:...|   |    4|          |          |         |   12312|     0|           0|xslF345X

In [10]:
spark.spark.sql(""" 
                SELECT count(*) FROM companyFacts
                """).show()

+--------+
|count(1)|
+--------+
|    1000|
+--------+



In [12]:
spark.sparkStop()

In [7]:
from source.scraper import get_spy500_formWiki


wiki = get_spy500_formWiki()

In [9]:
from source.scraper import get_StockPrices
import pandas as pd

df = get_StockPrices('NVDA', startSelect='2020-01-02')
df

,adjClose,date,ticker,Quarter,meanADJclose
0,5.973634,2020-01-02,NVDA,Q1-2020,6.285297
1,5.878020,2020-01-03,NVDA,Q1-2020,6.285297
2,5.902669,2020-01-06,NVDA,Q1-2020,6.285297
3,5.974131,2020-01-07,NVDA,Q1-2020,6.285297
4,5.985336,2020-01-08,NVDA,Q1-2020,6.285297
...,...,...,...,...,...
1159,109.019997,2024-08-12,NVDA,Q3-2024,117.722647
1160,116.139999,2024-08-13,NVDA,Q3-2024,117.722647
1161,118.080002,2024-08-14,NVDA,Q3-2024,117.722647
1162,122.860001,2024-08-15,NVDA,Q3-2024,117.722647


In [7]:
get_companyfacts('CIK0000320193')

,finType,label,description,end,val,accn,fy,fp,form,filed,frame,endFormat,start,startFormat,diffDate,monthWindow,yearMonthDay
0,AccountsPayable,Accounts Payable (Deprecated 2009-01-31),Carrying value as of the balance sheet date of...,2008-09-27,5.520000e+09,0001193125-09-153165,2009,Q3,10-Q,2009-07-22,CY2008Q3I,2008-09-27,NaN,NaT,NaN,NaN,20080927
1,AccountsPayable,Accounts Payable (Deprecated 2009-01-31),Carrying value as of the balance sheet date of...,2009-06-27,4.854000e+09,0001193125-09-153165,2009,Q3,10-Q,2009-07-22,CY2009Q2I,2009-06-27,NaN,NaT,NaN,NaN,20090627
2,AccountsPayableCurrent,"Accounts Payable, Current",Carrying value as of the balance sheet date of...,2008-09-27,5.520000e+09,0001193125-09-214859,2009,FY,10-K,2009-10-27,NaN,2008-09-27,NaN,NaT,NaN,NaN,20080927
3,AccountsPayableCurrent,"Accounts Payable, Current",Carrying value as of the balance sheet date of...,2008-09-27,5.520000e+09,0001193125-10-012091,2009,FY,10-K/A,2010-01-25,CY2008Q3I,2008-09-27,NaN,NaT,NaN,NaN,20080927
4,AccountsPayableCurrent,"Accounts Payable, Current",Carrying value as of the balance sheet date of...,2009-09-26,5.601000e+09,0001193125-09-214859,2009,FY,10-K,2009-10-27,NaN,2009-09-26,NaN,NaT,NaN,NaN,20090926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22738,IncrementalCommonSharesAttributableToShareBase...,Incremental Common Shares Attributable to Dilu...,Additional shares included in the calculation ...,2024-03-30,5.885300e+07,0000320193-24-000069,2024,Q2,10-Q,2024-05-03,CY2024Q1,2024-03-30,2023-12-31,2023-12-31,90.0,3.0,20240330
22739,IncrementalCommonSharesAttributableToShareBase...,Incremental Common Shares Attributable to Dilu...,Additional shares included in the calculation ...,2024-06-29,6.212800e+07,0000320193-24-000081,2024,Q3,10-Q,2024-08-02,NaN,2024-06-29,2023-10-01,2023-10-01,272.0,9.0,20240629
22740,IncrementalCommonSharesAttributableToShareBase...,Incremental Common Shares Attributable to Dilu...,Additional shares included in the calculation ...,2024-06-29,6.065400e+07,0000320193-24-000081,2024,Q3,10-Q,2024-08-02,CY2024Q2,2024-06-29,2024-03-31,2024-03-31,90.0,3.0,20240629
22741,OtherAssetsMiscellaneousNoncurrent,"Other Assets, Miscellaneous, Noncurrent",Amount of other miscellaneous assets expected ...,2022-09-24,3.905300e+10,0000320193-23-000106,2023,FY,10-K,2023-11-03,CY2022Q3I,2022-09-24,NaN,NaT,NaN,NaN,20220924


In [2]:
from source.scraper import get_companyfacts
#https://data.sec.gov/api/xbrl/companyfacts/CIK0000320193.json
mergedDF = get_companyfacts('CIK0001045810') #2016-07-31
mergedDF[mergedDF['val'] == '2016-07-31']

,finType,label,description,start,end,val,accn,fy,fp,form,filed,frame,startFormat,endFormat,diffDate,monthWindow,yearMonthDay
0,AcceleratedShareRepurchaseProgramAdjustment,"Accelerated Share Repurchase Program, Adjustment",The amount needed to adjust previously recorde...,2016-05-02,2016-07-31,9.000000e+06,0001045810-16-000300,2016,Q2,10-Q,2016-08-23,CY2016Q2,2016-05-02,2016-07-31,90.0,3.0,20160731
1,AcceleratedShareRepurchaseProgramAdjustment,"Accelerated Share Repurchase Program, Adjustment",The amount needed to adjust previously recorde...,2016-02-01,2016-10-30,9.000000e+06,0001045810-16-000353,2016,Q3,10-Q,2016-11-22,NaN,2016-02-01,2016-10-30,272.0,9.0,20161030
2,AcceleratedShareRepurchasesFinalPricePaidPerShare,"Accelerated Share Repurchases, Final Price Pai...",Final price paid per share for the purchase of...,2015-01-26,2015-10-25,2.163000e+01,0001045810-15-000173,2015,Q3,10-Q,2015-11-18,NaN,2015-01-26,2015-10-25,272.0,9.0,20151025
3,AcceleratedShareRepurchasesFinalPricePaidPerShare,"Accelerated Share Repurchases, Final Price Pai...",Final price paid per share for the purchase of...,2016-02-01,2016-07-31,4.206000e+01,0001045810-16-000300,2016,Q2,10-Q,2016-08-23,NaN,2016-02-01,2016-07-31,181.0,6.0,20160731
4,AcceleratedShareRepurchasesFinalPricePaidPerShare,"Accelerated Share Repurchases, Final Price Pai...",Final price paid per share for the purchase of...,2016-02-01,2016-10-30,4.206000e+01,0001045810-16-000353,2016,Q3,10-Q,2016-11-22,NaN,2016-02-01,2016-10-30,272.0,9.0,20161030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23308,ProceedsFromStockPlans,Proceeds from Stock Plans,The cash inflow associated with the amount rec...,2022-01-31,2023-01-29,3.550000e+08,0001045810-24-000029,2024,FY,10-K,2024-02-21,CY2022,2022-01-31,2023-01-29,363.0,12.0,20230129
23309,ProceedsFromStockPlans,Proceeds from Stock Plans,The cash inflow associated with the amount rec...,2023-01-30,2023-04-30,2.460000e+08,0001045810-24-000124,2025,Q1,10-Q,2024-05-29,CY2023Q1,2023-01-30,2023-04-30,90.0,3.0,20230430
23310,ProceedsFromStockPlans,Proceeds from Stock Plans,The cash inflow associated with the amount rec...,2023-01-30,2024-01-28,4.030000e+08,0001045810-24-000029,2024,FY,10-K,2024-02-21,CY2023,2023-01-30,2024-01-28,363.0,12.0,20240128
23311,ProceedsFromStockPlans,Proceeds from Stock Plans,The cash inflow associated with the amount rec...,2024-01-29,2024-04-28,2.850000e+08,0001045810-24-000124,2025,Q1,10-Q,2024-05-29,CY2024Q1,2024-01-29,2024-04-28,90.0,3.0,20240428


In [8]:
mergedDF.columns

Index(['finType', 'label', 'description', 'start', 'end', 'val', 'accn', 'fy',
       'fp', 'form', 'filed', 'frame', 'startFormat', 'endFormat', 'diffDate',
       'monthWindow', 'yearMonthDay'],
      dtype='object')

In [9]:
desired_order = [
    'finType', 'label', 'description', 'end', 'val', 'accn', 'fy', 'fp', 
    'form', 'filed', 'frame', 'endFormat', 'start', 'startFormat', 
    'diffDate', 'monthWindow', 'yearMonthDay'
]

mergedDF[desired_order]

,finType,label,description,end,val,accn,fy,fp,form,filed,frame,endFormat,start,startFormat,diffDate,monthWindow,yearMonthDay
0,AcceleratedShareRepurchaseProgramAdjustment,"Accelerated Share Repurchase Program, Adjustment",The amount needed to adjust previously recorde...,2016-07-31,9.000000e+06,0001045810-16-000300,2016,Q2,10-Q,2016-08-23,CY2016Q2,2016-07-31,2016-05-02,2016-05-02,90.0,3.0,20160731
1,AcceleratedShareRepurchaseProgramAdjustment,"Accelerated Share Repurchase Program, Adjustment",The amount needed to adjust previously recorde...,2016-10-30,9.000000e+06,0001045810-16-000353,2016,Q3,10-Q,2016-11-22,NaN,2016-10-30,2016-02-01,2016-02-01,272.0,9.0,20161030
2,AcceleratedShareRepurchasesFinalPricePaidPerShare,"Accelerated Share Repurchases, Final Price Pai...",Final price paid per share for the purchase of...,2015-10-25,2.163000e+01,0001045810-15-000173,2015,Q3,10-Q,2015-11-18,NaN,2015-10-25,2015-01-26,2015-01-26,272.0,9.0,20151025
3,AcceleratedShareRepurchasesFinalPricePaidPerShare,"Accelerated Share Repurchases, Final Price Pai...",Final price paid per share for the purchase of...,2016-07-31,4.206000e+01,0001045810-16-000300,2016,Q2,10-Q,2016-08-23,NaN,2016-07-31,2016-02-01,2016-02-01,181.0,6.0,20160731
4,AcceleratedShareRepurchasesFinalPricePaidPerShare,"Accelerated Share Repurchases, Final Price Pai...",Final price paid per share for the purchase of...,2016-10-30,4.206000e+01,0001045810-16-000353,2016,Q3,10-Q,2016-11-22,NaN,2016-10-30,2016-02-01,2016-02-01,272.0,9.0,20161030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23308,ProceedsFromStockPlans,Proceeds from Stock Plans,The cash inflow associated with the amount rec...,2023-01-29,3.550000e+08,0001045810-24-000029,2024,FY,10-K,2024-02-21,CY2022,2023-01-29,2022-01-31,2022-01-31,363.0,12.0,20230129
23309,ProceedsFromStockPlans,Proceeds from Stock Plans,The cash inflow associated with the amount rec...,2023-04-30,2.460000e+08,0001045810-24-000124,2025,Q1,10-Q,2024-05-29,CY2023Q1,2023-04-30,2023-01-30,2023-01-30,90.0,3.0,20230430
23310,ProceedsFromStockPlans,Proceeds from Stock Plans,The cash inflow associated with the amount rec...,2024-01-28,4.030000e+08,0001045810-24-000029,2024,FY,10-K,2024-02-21,CY2023,2024-01-28,2023-01-30,2023-01-30,363.0,12.0,20240128
23311,ProceedsFromStockPlans,Proceeds from Stock Plans,The cash inflow associated with the amount rec...,2024-04-28,2.850000e+08,0001045810-24-000124,2025,Q1,10-Q,2024-05-29,CY2024Q1,2024-04-28,2024-01-29,2024-01-29,90.0,3.0,20240428


In [3]:
mergedDF[mergedDF['val'] == '2016-07-31']

,finType,label,description,start,end,val,accn,fy,fp,form,filed,frame,startFormat,endFormat,diffDate,monthWindow,yearMonthDay


In [4]:
mergedDF['val'].astype(float)

0        9.000000e+06
1        9.000000e+06
2        2.163000e+01
3        4.206000e+01
4        4.206000e+01
             ...     
23308    3.550000e+08
23309    2.460000e+08
23310    4.030000e+08
23311    2.850000e+08
23312    1.200000e+09
Name: val, Length: 23313, dtype: float64

In [5]:
len(mergedDF.columns)

17

In [16]:
mergedDF['accn'].value_counts()

0001193125-13-416534    578
0001193125-15-023732    569
0001193125-14-383437    569
0001193125-10-238044    568
0000320193-17-000070    558
                       ... 
0001193125-10-012091    208
0000320193-24-000006    194
0001193125-09-153165    172
0001193125-10-088957    171
0001193125-10-012085    140
Name: accn, Length: 64, dtype: int64

In [11]:
"""
https://www.sec.gov/ix?doc=/Archives/edgar/data/0000320193/000114036124024352/ef20028273_8k.htm


https://data.sec.gov/api/xbrl/companyconcept/CIK0000320193/us-gaap/AccountsPayableCurrent.json - info about reports

https://data.sec.gov/api/xbrl/companyfacts/CIK0000320193.json

/0001140361|24|024352/
accessionNumber - 0001140361-24-024352

"""

'\nhttps://www.sec.gov/ix?doc=/Archives/edgar/data/0000320193/000114036124024352/ef20028273_8k.htm\n\n\nhttps://data.sec.gov/api/xbrl/companyconcept/CIK0000320193/us-gaap/AccountsPayableCurrent.json - info about reports\n\nhttps://data.sec.gov/api/xbrl/companyfacts/CIK0000320193.json\n\n/0001140361|24|024352/\naccessionNumber - 0001140361-24-024352\n\n'

In [12]:
from source.scraper import generate_CIK_TICKER

sec_ticker = generate_CIK_TICKER()
sec_ticker

,cik_str,ticker,title,cik10D
0,320193,AAPL,Apple Inc.,0000320193
1,789019,MSFT,MICROSOFT CORP,0000789019
2,1045810,NVDA,NVIDIA CORP,0001045810
3,1652044,GOOGL,Alphabet Inc.,0001652044
4,1018724,AMZN,AMAZON COM INC,0001018724
...,...,...,...,...
10206,2001699,HPAIW,Helport AI Ltd,0002001699
10207,1991946,CGBSW,Crown LNG Holdings Ltd,0001991946
10208,1904616,BAKR,Baker Global Asset Management Inc.,0001904616
10209,808219,MHGUP,MERITAGE HOSPITALITY GROUP INC,0000808219


In [3]:
from source.scraper import get_SEC_filings


ticker = 'AAPL'

filings = get_SEC_filings('CIK0000320193', 'AAPL')

filings

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription,fileURL,yearMonthDay,ticket
0,0000320193-24-000086,2024-08-13,2024-08-09,2024-08-13T19:02:12.000Z,,4,,,,5031,0,0,xslF345X05/wk-form4_1723590126.xml,FORM 4,https://www.sec.gov/Archives/edgar/data/000032...,20240809,AAPL
1,0000320193-24-000084,2024-08-07,2024-08-05,2024-08-07T18:30:15.000Z,,4,,,,8270,0,0,xslF345X05/wk-form4_1723069808.xml,FORM 4,https://www.sec.gov/Archives/edgar/data/000032...,20240805,AAPL
2,0001950047-24-005824,2024-08-05,,2024-08-05T16:32:46.000Z,33,144,001-36743,241175294,,8541,0,0,xsl144X01/primary_doc.xml,,https://www.sec.gov/Archives/edgar/data/000032...,20240805,AAPL
3,0000320193-24-000081,2024-08-02,2024-06-29,2024-08-01T18:03:34.000Z,34,10-Q,001-36743,241168182,,5372771,1,1,aapl-20240629.htm,10-Q,https://www.sec.gov/Archives/edgar/data/000032...,20240629,AAPL
4,0000320193-24-000080,2024-08-01,2024-08-01,2024-08-01T16:30:26.000Z,34,8-K,001-36743,241167232,"2.02,9.01",437770,1,1,aapl-20240801.htm,8-K,https://www.sec.gov/Archives/edgar/data/000032...,20240801,AAPL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0001181431-14-005066,2014-02-04,2014-02-01,2014-02-04T18:33:20.000Z,,4,,,,6799,0,0,xslF345X03/rrd401118.xml,14.02.01 GORE FORM 4 - RSU VESTING,https://www.sec.gov/Archives/edgar/data/000032...,20140201,AAPL
996,0001181431-14-005065,2014-02-04,2014-02-01,2014-02-04T18:33:00.000Z,,4,,,,7497,0,0,xslF345X03/rrd401119.xml,14.02.01 IGER FORM 4 - RSU VESTING,https://www.sec.gov/Archives/edgar/data/000032...,20140201,AAPL
997,0001181431-14-005064,2014-02-04,2014-02-01,2014-02-04T18:32:22.000Z,,4,,,,6811,0,0,xslF345X03/rrd401122.xml,14.02.01 JUNG FORM 4 - RSU VESTING,https://www.sec.gov/Archives/edgar/data/000032...,20140201,AAPL
998,0001181431-14-005063,2014-02-04,2014-02-01,2014-02-04T18:32:03.000Z,,4,,,,7514,0,0,xslF345X03/rrd401124.xml,14.02.01 LEVINSON FORM 4 - RSU VESTING,https://www.sec.gov/Archives/edgar/data/000032...,20140201,AAPL


In [20]:
filings['reportDate'][2]

''

In [1]:
import pyspark.pandas as ps

c:\Users\proso\AppData\Local\Programs\Python\Python310\lib\site-packages\pyspark\pandas\__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(
